In [1]:
import duckdb

# puzzle input goes here
input = """
"""

In [2]:
duckdb.sql("""
    WITH input AS (
        SELECT '%s' AS input
    ), t AS (
        SELECT
            UNNEST(STRING_SPLIT(input, '\n')) i,
            CAST(REGEXP_EXTRACT(STRING_SPLIT(i, ':')[1], '([0-9]+)') AS INT) card,
            CAST(REGEXP_EXTRACT_ALL(STRING_SPLIT(STRING_SPLIT(i, ':')[2], '|')[1], '([0-9]+)') AS INT[]) win,
            CAST(REGEXP_EXTRACT_ALL(STRING_SPLIT(STRING_SPLIT(i, ':')[2], '|')[2], '([0-9]+)') AS INT[]) hand
        FROM input
    ), matches AS (
        SELECT card, LENGTH(ARRAY_INTERSECT(win, hand)) matches
        FROM t
    )
    SELECT CAST(SUM(2^(matches - 1)) AS INT) ans
    FROM matches
    WHERE matches > 0
""" % input)

┌───────┐
│  ans  │
│ int32 │
├───────┤
│ 26443 │
└───────┘

In [3]:
duckdb.sql("""
    WITH RECURSIVE input AS (
        SELECT '%s' AS input
    ), t AS (
        SELECT
            UNNEST(STRING_SPLIT(input, '\n')) i,
            CAST(REGEXP_EXTRACT(STRING_SPLIT(i, ':')[1], '([0-9]+)') AS INT) card,
            CAST(REGEXP_EXTRACT_ALL(STRING_SPLIT(STRING_SPLIT(i, ':')[2], '|')[1], '([0-9]+)') AS INT[]) win,
            CAST(REGEXP_EXTRACT_ALL(STRING_SPLIT(STRING_SPLIT(i, ':')[2], '|')[2], '([0-9]+)') AS INT[]) hand
        FROM input
    ), matches AS (
        SELECT card, LENGTH(ARRAY_INTERSECT(win, hand)) matches
        FROM t
    ), copies AS (
        SELECT card, matches
        FROM matches
        UNION ALL
        SELECT matches.card, matches.matches
        FROM copies, matches
        WHERE matches.card - copies.card BETWEEN 1 AND copies.matches
    )
    SELECT COUNT(*) AS ans
    FROM copies
""" % input)

┌─────────┐
│   ans   │
│  int64  │
├─────────┤
│ 6284877 │
└─────────┘